In [2]:
# importing libraries
import pandas as pd

Since the dataset is split into **multiple files**, I'm going to read file-by-file and save them to a list so that I can concatenate them together for later analysis.

In [9]:
pieces = []
for year in range(1880, 2024):
    path = f'datasets/yob{year}.txt'
    frame = pd.read_csv(path, sep=',', names=['name', 'sex', 'births'])
    # assign the year to the frame
    frame['year'] = year
    # append the frame to pieces
    pieces.append(frame)

# concatenate the pieces into a single DataFrame
names = pd.concat(pieces, ignore_index=True)
names.head()

,name,sex,births,year
0,Mary,F,7065,1880
1,Anna,F,2604,1880
2,Emma,F,2003,1880
3,Elizabeth,F,1939,1880
4,Minnie,F,1746,1880
